# PPI classification

See https://docs.dgl.ai/generated/dgl.data.PPIDataset.html#dgl.data.PPIDataset
See https://graphneural.network/datasets/#ppi

In [150]:
import json
from dataclasses import dataclass

import networkx as nx
import numpy as np
import pandas as pd
from IPython.core.display_functions import clear_output, display
from igraph import Graph, Vertex
from networkx.readwrite import json_graph
from scipy.io import loadmat
from scipy.sparse import csc_matrix
from stellargraph import StellarGraph
from tensorflow import keras

from lib.DataSet import DataSet
from lib.ProjectGraph import balance, largest_component


## Methods and functions

### Word2Vec

In [165]:
from stellargraph.layer import Node2Vec, link_classification
from stellargraph.mapper import Node2VecLinkGenerator, Node2VecNodeGenerator
from stellargraph.data import BiasedRandomWalk, UnsupervisedSampler


class Word2VecResult:

    embeddings = None

    def __init__(self, embeddings):
        self.embeddings = embeddings

def word2vec_run(graph: StellarGraph, **kwargs) -> Word2VecResult:

    walker_config = {

        # Defines default random walker config (used during testing)
        'n': 5,            # n         --> total number of random walks per node
        'length': 25,        # length    --> Maximum length of each random walk
        'p': 1.0,           # p         --> Defines probability, 1/p, of returning to source node
        'q': 1.0,           # q         --> Defines probability, 1/q, for moving to a node away from the source node

        # overrides previous configurations
        **kwargs
    }

    # print("Q should be {}, but is {}".format(kwargs['q'], walker_config['q']))

    node2vec_config = {

        # Node2Vec link generator config
        'batch_size': 50,
        'epochs': 2,
        'embedding_size': 128,

        'learning_rate': 1e-3,
        'epsilon': 1e-7,
        'momentum': 0.01,
        'verbose': 0,

        # overrides previous configurations
        **kwargs
    }

    # configure a random walker
    walker = BiasedRandomWalk(
        graph,
        n=walker_config['n'],
        length=walker_config['length'],
        p=walker_config['p'],
        q=walker_config['q'],
    )

    # sampler
    unsupervised_samples = UnsupervisedSampler(graph, nodes=list(graph.nodes()), walker=walker)

    #
    generator = Node2VecLinkGenerator(graph, node2vec_config['batch_size'])

    node2vec = Node2Vec(node2vec_config['embedding_size'], generator=generator)

    x_inp, x_out = node2vec.in_out_tensors()

    prediction = link_classification(
        output_dim=1, output_act="sigmoid", edge_embedding_method="dot"
    )(x_out)

    sgd_optimizer = keras.optimizers.SGD(
        learning_rate=node2vec_config['learning_rate'], momentum=node2vec_config['momentum'], nesterov=False, name="SGD"
    )
    # adam_optimizer = keras.optimizers.Adam(learning_rate=node2vec_config['learning_rate'], epsilon=node2vec_config['epsilon'])
    adam_optimizer = keras.optimizers.Adam(learning_rate=node2vec_config['learning_rate'])

    model = keras.Model(inputs=x_inp, outputs=prediction)
    model.compile(
        optimizer=adam_optimizer,
        loss=keras.losses.binary_crossentropy,
        metrics=[keras.metrics.binary_accuracy],
    )

    history = model.fit(
        generator.flow(unsupervised_samples),
        epochs=node2vec_config['epochs'],
        verbose=node2vec_config['verbose'],
        use_multiprocessing=False,
        workers=4,
        shuffle=True,
    )

    #
    # NODE PREDICTION MODEL
    #

    x_inp_src = x_inp[0]
    x_out_src = x_out[0]
    embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)

    node_ids = graph.nodes()

    node_gen = Node2VecNodeGenerator(graph, node2vec_config['batch_size']).flow(node_ids)
    node_embeddings = embedding_model.predict(node_gen, workers=4, verbose=1)

    return Word2VecResult(
        embeddings=node_embeddings
    )

### Classification

In [249]:
from sklearn.linear_model import LogisticRegression


@dataclass
class PredictionPerformance:

    description: str
    accuracy: float
    f1_micro: float
    f1_macro: float


def predict_node_classes(X, y, description='Node prediction results', scoring="accuracy", split=1) -> PredictionPerformance:

    # X will hold the 128-dimensional input features
    # X = node_embeddings
    # y holds the corresponding target values
    # y = np.array(labels)

    from sklearn.model_selection import train_test_split

    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7/split, test_size=0.3/split)
    # print(
    #     "Array shapes:\n X_train = {}\n y_train = {}\n X_test = {}\n y_test = {}".format(
    #         X_train.shape, y_train.shape, X_test.shape, y_test.shape
    #     )
    # )

    # training

    from sklearn.linear_model import LogisticRegressionCV

    clf = LogisticRegressionCV(
        Cs=10, cv=10, scoring=scoring, penalty="l2", solver="liblinear", verbose=False, multi_class="ovr", max_iter=300
    )
    # clf = LogisticRegression(
    #     penalty="l2", solver="liblinear", verbose=False, multi_class="ovr", max_iter=100
    # )
    clf.fit(X_train, y_train)

    # predicting

    y_pred = clf.predict(X_test)

    # scoring
    from sklearn.metrics import accuracy_score, f1_score

    return PredictionPerformance(
        description=description,
        accuracy=accuracy_score(y_test, y_pred),
        f1_micro=f1_score(y_test, y_pred, average='micro'),
        f1_macro=f1_score(y_test, y_pred, average='macro')
    )

## GraphSAGE PPI dataset

### Data import

In [5]:
def get_ppi_data(only_largest_component: bool = False) -> StellarGraph:

    # load data
    json_file = open('./data/ppi/ppi-G.json')
    json_data = json.load(json_file)

    # load into nxGraph
    graph: nx.Graph = json_graph.node_link_graph(json_data)

    # convert into stellargraph
    stellar_graph = StellarGraph.from_networkx(graph)

    # for this project, we want only the largest component
    if only_largest_component:
        largest_component_nodes = next(stellar_graph.connected_components())
        stellar_graph = stellar_graph.subgraph(largest_component_nodes)

    return stellar_graph

In [6]:
def get_ppi_labels(graph: StellarGraph) -> np.array:

    # reading the json file
    class_map_file = open('./data/ppi/ppi-class_map.json')
    class_map_dict = json.load(class_map_file)

    # generating a label matrix
    labels = np.array([class_map_dict[str(n)] for n in graph.nodes()])

    return labels

### Experiments

In [7]:
n = 100
length = 7

ppi_graph = get_ppi_data(only_largest_component=True)
ppi_results: Word2VecResult = word2vec_run(ppi_graph, n=n, length=length, p=0.5, q=9.0)

link_classification: using 'dot' method to combine node embeddings into edge embeddings
Epoch 1/2
83520/83520 [==============================] - 278s 3ms/step - loss: 0.6163 - binary_accuracy: 0.6445
Epoch 2/2
70/70 [==============================] - 0s 2ms/step


In [9]:
# load label data
ppi_labels = get_ppi_labels(ppi_graph)

In [8]:
# balance the categories (true/false)
balanced_embeddings, balanced_labels = balance(ppi_results.embeddings, ppi_labels)

# measure accuracy
accuracy = []
for i in np.arange(1,100):
    classification_performance = predict_node_classes(balanced_embeddings, balanced_labels)
    accuracy = np.append(accuracy, [classification_performance.accuracy])

np.mean(accuracy)

IndexError: index 147256 is out of bounds for axis 0 with size 3480

## Node2Vec PPI dataset

### Data Import

In [90]:
def get_hosap_labels(only_largest_category: bool = False) -> np.array:

    # load adjacency matrix data
    hs = loadmat('./data/Homo_sapiens.mat', matlab_compatible=True)

    group = np.array(csc_matrix(hs['group'], dtype=float).toarray())

    if only_largest_category:

        # find out index of largest group
        group_sizes = np.sum(group, axis=0)
        biggest_group_index = None
        biggest_group_size = 0
        for i, size in enumerate(group_sizes):
            if size > biggest_group_size:
                biggest_group_size = size
                biggest_group_index = i

        # resize groups
        group = group[:,biggest_group_index]

    return group

In [199]:
def get_hosap_graph(only_largest_component: bool = False) -> (Graph, StellarGraph, np.array):

    # load adjacency matrix data
    data = loadmat('./data/Homo_sapiens.mat', matlab_compatible=False)

    network = np.array(csc_matrix(data['network'], dtype=float).toarray())
    # group = np.array(csc_matrix(data['group'], dtype=int).toarray())
    graph_labels = get_hosap_labels(only_largest_category=True)

    # network_graph: Graph = Graph.Adjacency(csc_matrix(hs['network'], dtype=float), mode="undirected")
    nw_graph: Graph = Graph.Adjacency(network, mode="undirected")

    label_dict = []
    label_index = []

    # attach labels
    for v in nw_graph.vs:
        v: Vertex = v
        v_id = v.index
        v_label = graph_labels[v_id]
        # print(v_label)
        nw_graph.vs[v_id]['node_id'] = v_id
        nw_graph.vs[v_id]['labels'] = [v_label]
        label_dict.append(str(v_label))
        label_index.append(str(v_id))
        # break

    if only_largest_component:
        nw_graph = largest_component(nw_graph)

    label_values = nw_graph.vs.get_attribute_values('labels')

    # prepare graph data format
    igraph_edges = np.array(nw_graph.get_edgelist())
    stellar_edges = pd.DataFrame(data=igraph_edges, columns=['source', 'target'])
    # stellartest = StellarGraph(edges=simple_edges, is_directed=False, nodes=group)
    stellar_graph = StellarGraph.from_networkx(nw_graph.to_networkx(), node_features="labels")
    # stellar_graph = StellarGraph(edges=stellar_edges, is_directed=False, nodes=label_values)

    # print(nw_graph.vs.attribute_names())

    # for this project, we want only the largest component
    # if only_largest_component:
    #     largest_component_nodes = next(stellar_graph.connected_components())
    #     stellar_graph = stellar_graph.subgraph(largest_component_nodes)

    # construct label series
    label_series = pd.Series(data=label_dict, index=label_index, name='category')

    # return nw_graph, stellar_graph, np.array(label_values)[:,0]
    return nw_graph, stellar_graph, label_series

In [200]:
igraph, stellargraph, labels = get_hosap_graph(only_largest_component=True)
# print(igraph.summary())
# print(stellargraph.info())
# display(pd.DataFrame(labels, columns=['class']))
labels

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
3885    0.0
3886    0.0
3887    0.0
3888    1.0
3889    0.0
Name: category, Length: 3890, dtype: object

In [193]:
pd.Series(igraph.vs.get_attribute_values('node_id'), name='node id')

0          0
1          1
2          2
3          3
4          4
        ... 
3847    3885
3848    3886
3849    3887
3850    3888
3851    3889
Name: node id, Length: 3852, dtype: int64

In [186]:
pd.Series(labels, name='class')

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
3847    0.0
3848    0.0
3849    0.0
3850    1.0
3851    0.0
Name: class, Length: 3852, dtype: float64

In [152]:
n = 20
length = 3

hosap_igraph, hosap_stellar, labels = get_hosap_graph()
hosap_results: Word2VecResult = word2vec_run(hosap_stellar, n=n, length=length, p=0.5, q=1.0)

link_classification: using 'dot' method to combine node embeddings into edge embeddings
Epoch 1/2
6224/6224 [==============================] - 24s 4ms/step - loss: 0.6565 - binary_accuracy: 0.5815
Epoch 2/2
78/78 [==============================] - 0s 2ms/step


In [158]:
hosap_results: Word2VecResult = word2vec_run(hosap_stellar, n=n, length=length, p=0.5, q=1.0, embedding_size=16)

link_classification: using 'dot' method to combine node embeddings into edge embeddings
Epoch 1/2
6224/6224 [==============================] - 7s 1ms/step - loss: 0.6970 - binary_accuracy: 0.5370
Epoch 2/2
78/78 [==============================] - 0s 2ms/step


In [153]:
hosap_results: Word2VecResult = word2vec_run(hosap_stellar, n=n, length=length, p=0.5, q=1.0, embedding_size=32)

link_classification: using 'dot' method to combine node embeddings into edge embeddings
Epoch 1/2
6224/6224 [==============================] - 8s 1ms/step - loss: 0.6854 - binary_accuracy: 0.5541
Epoch 2/2
78/78 [==============================] - 0s 2ms/step


In [154]:
hosap_results: Word2VecResult = word2vec_run(hosap_stellar, n=n, length=length, p=0.5, q=1.0, embedding_size=64)

link_classification: using 'dot' method to combine node embeddings into edge embeddings
Epoch 1/2
6224/6224 [==============================] - 11s 2ms/step - loss: 0.6714 - binary_accuracy: 0.5691
Epoch 2/2
78/78 [==============================] - 0s 2ms/step


In [155]:
hosap_results: Word2VecResult = word2vec_run(hosap_stellar, n=n, length=length, p=0.5, q=1.0, embedding_size=128)

link_classification: using 'dot' method to combine node embeddings into edge embeddings
Epoch 1/2
6224/6224 [==============================] - 25s 4ms/step - loss: 0.6571 - binary_accuracy: 0.5817
Epoch 2/2
78/78 [==============================] - 0s 2ms/step


In [156]:
hosap_results: Word2VecResult = word2vec_run(hosap_stellar, n=n, length=length, p=0.5, q=1.0, embedding_size=256)

link_classification: using 'dot' method to combine node embeddings into edge embeddings
Epoch 1/2
6224/6224 [==============================] - 66s 10ms/step - loss: 0.6445 - binary_accuracy: 0.5911
Epoch 2/2
78/78 [==============================] - 0s 2ms/step


In [157]:
hosap_results: Word2VecResult = word2vec_run(hosap_stellar, n=n, length=length, p=0.5, q=1.0, embedding_size=512)

link_classification: using 'dot' method to combine node embeddings into edge embeddings
Epoch 1/2
5640/6224 [==========================>...] - ETA: 14s - loss: 0.6466 - binary_accuracy: 0.5908

KeyboardInterrupt: 

In [112]:
hosap_results: Word2VecResult = word2vec_run(hosap_stellar, n=n, length=length, p=0.5, q=9.0)

link_classification: using 'dot' method to combine node embeddings into edge embeddings
Epoch 1/2
93360/93360 [==============================] - 424s 5ms/step - loss: 0.5658 - binary_accuracy: 0.6856
Epoch 2/2
78/78 [==============================] - 0s 2ms/step


Evaluating performance of last run:

In [127]:
# balance the categories (true/false)
balanced_embeddings, balanced_labels = balance(hosap_results.embeddings, labels[:,0])

# measure accuracy
accuracy = []
f1_macro = []
f1_micro = []
for i in np.arange(1,100):
    classification_performance = predict_node_classes(balanced_embeddings, balanced_labels)
    accuracy = np.append(accuracy, [classification_performance.accuracy])
    f1_macro = np.append(f1_macro, [classification_performance.f1_macro])
    f1_micro = np.append(f1_micro, [classification_performance.f1_micro])

In [128]:
np.mean(accuracy)

0.715233139475564

In [129]:
np.mean(f1_micro)

0.715233139475564

In [130]:
np.mean(f1_macro)

0.7133852091359285

In [250]:
def experiment(graph: StellarGraph, graph_labels, n_values=[20], length_values=[3], q_values=[1.0], p_values=[1.0], embedding_size=128, classification_repeats=100, split=1) -> pd.DataFrame:
    results = None
    for n in n_values:
        for length in length_values:
            for q in q_values:
                for p in p_values:

                    # calculate embeddings
                    word2vec_results: Word2VecResult = word2vec_run(graph, n=n, length=length, p=p, q=q, embedding_size=embedding_size)

                    # experiment_balanced_embeddings, experiment_balanced_labels = balance(word2vec_results.embeddings, graph_labels)
                    experiment_balanced_embeddings = word2vec_results.embeddings
                    experiment_balanced_labels = graph_labels

                    # predict and evaluate performance
                    classification_accuracy = []
                    classification_f1_macro = []
                    classification_f1_micro = []
                    for i in np.arange(1,classification_repeats):
                        performance = predict_node_classes(experiment_balanced_embeddings, experiment_balanced_labels, scoring="f1_macro", split=split)
                        classification_accuracy = np.append(classification_accuracy, [performance.accuracy])
                        classification_f1_macro = np.append(classification_f1_macro, [performance.f1_macro])
                        classification_f1_micro = np.append(classification_f1_micro, [performance.f1_micro])

                    # summarise performance
                    avg_accuracy = np.mean(classification_accuracy)
                    std_accuracy = np.std(classification_accuracy)
                    avg_f1_micro = np.mean(classification_f1_micro)
                    std_f1_micro = np.std(classification_f1_micro)
                    avg_f1_macro = np.mean(classification_f1_macro)
                    std_f1_macro = np.std(classification_f1_macro)

                    # collect metrics
                    metrics_data_row = np.array([[embedding_size, p, q, avg_accuracy, std_accuracy, avg_f1_micro, std_f1_micro, avg_f1_macro, std_f1_macro]])
                    if results is None:
                        results = metrics_data_row
                    else:
                        results = np.append(results, metrics_data_row, axis=0)

                    results_table = pd.DataFrame(data=results, columns=['embedding size', 'p', 'q', 'accuracy (mean)', 'accuracy (std)', 'f1_micro (mean)', 'f1_micro (std)', 'f1_macro (mean)', 'f1_macro (std)'])

                    clear_output(wait=True)
                    display(results_table)

    return results_table

In [161]:
# results = np.array([['a', 'b', 'c']])
# results = np.append(results, np.array([['a', 'b', 'c']]), axis = 0)
# results = pd.DataFrame(data=results, columns=['aa', 'bb', 'cc'])
# results

,aa,bb,cc
0,a,b,c
1,a,b,c


In [201]:
hosap_igraph, hosap_stellar, hosap_labels = get_hosap_graph()

In [204]:
experiment_results_table = experiment(hosap_stellar, hosap_labels, embedding_size=32)

clear_output(wait=True)
display(experiment_results_table)

,embedding size,p,q,accuracy (mean),accuracy (std),f1_micro (mean),f1_micro (std),f1_macro (mean),f1_macro (std)
0,32.0,1.0,1.0,0.949562,0.003578,0.949562,0.003578,0.487063,0.000941


In [ ]:
experiment_results_table = experiment(hosap_stellar, hosap_labels, embedding_size=64)

clear_output(wait=True)
display(experiment_results_table)

In [ ]:
experiment_results_table = experiment(hosap_stellar, hosap_labels, embedding_size=128)

clear_output(wait=True)
display(experiment_results_table)

In [ ]:
experiment_results_table = experiment(hosap_stellar, hosap_labels, embedding_size=256)

clear_output(wait=True)
display(experiment_results_table)

In [ ]:
experiment_results_table = experiment(hosap_stellar, hosap_labels, embedding_size=512)

clear_output(wait=True)
display(experiment_results_table)

In [168]:
n_values = [20]
length_values = [3]
# q_values = np.array([0.2, 0.4, 0.6, 0.8, 1.0, 4.0, 7.0, 10.0])
# p_values = np.array([0.2, 0.4, 0.6, 0.8, 1.0, 4.0, 7.0, 10.0])
q_values = np.array([0.1, 0.3, 0.6, 1.0, 5.0, 10.0])
p_values = np.array([0.1, 0.3, 0.6, 1.0, 5.0, 10.0])

experiment_results_table = experiment(hosap_stellar, hosap_labels, n_values=n_values, length_values=length_values, p_values=p_values, q_values=q_values, embedding_size=32)

clear_output(wait=True)
display(experiment_results_table)

,embedding size,p,q,accuracy (mean),accuracy (std),f1_micro (mean),f1_micro (std),f1_macro (mean),f1_macro (std)
0,32.0,0.1,0.1,0.492297,0.028310,0.492297,0.028310,0.489071,0.028346
1,32.0,0.3,0.1,0.498827,0.032246,0.498827,0.032246,0.495062,0.032036
2,32.0,0.6,0.1,0.528415,0.027612,0.528415,0.027612,0.523650,0.027174
3,32.0,1.0,0.1,0.556729,0.030002,0.556729,0.030002,0.552496,0.029483
4,32.0,5.0,0.1,0.501020,0.027728,0.501020,0.027728,0.489884,0.027841
5,32.0,10.0,0.1,0.474696,0.031833,0.474696,0.031833,0.471186,0.032144
6,32.0,0.1,0.3,0.548515,0.032965,0.548515,0.032965,0.544348,0.033463
7,32.0,0.3,0.3,0.536119,0.033047,0.536119,0.033047,0.531382,0.033067
8,32.0,0.6,0.3,0.522702,0.028816,0.522702,0.028816,0.518826,0.029428
9,32.0,1.0,0.3,0.515356,0.028767,0.515356,0.028767,0.510722,0.029631


In [171]:
experiment_results_table.to_excel('./ppi-results-32.xlsx')
experiment_results_table.to_pickle('./ppi-results-32.pkl')
experiment_results_table.to_csv('./ppi-results-32.csv')
experiment_results_table.to_latex('./ppi-results-32.tex')

/var/folders/rr/p4lmjn152vj75lt21_txptlh0000gn/T/ipykernel_40687/3633795417.py:4: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  experiment_results_table.to_latex('./ppi-results-32.tex')


## Cora dataset

In [205]:
from stellargraph.datasets import datasets


def get_cora_graph() -> (StellarGraph, pd.Series):
    dataset = datasets.Cora()
    # display(HTML(dataset.description))
    graph, subjects = dataset.load(largest_connected_component_only=True)

    return graph, subjects

In [216]:
np.append(np.arange(0.1, 1.0, 0.2), np.arange(1.0, 10.0, 2.0))

array([0.1, 0.3, 0.5, 0.7, 0.9, 1. , 3. , 5. , 7. , 9. ])

In [248]:
cora_stellar, cora_labels = get_cora_graph()

n_values = [100]
length_values = [3]
p_values = np.append(np.arange(0.1, 1.0, 0.2), np.arange(1.0, 10.0, 2.0))
q_values = np.append(np.arange(0.1, 1.0, 0.2), np.arange(1.0, 10.0, 2.0))
# p_values = [1.0, 3.0, 5.0, 7.0]
# q_values = [1.0]
experiment_results_table = experiment(cora_stellar, cora_labels, q_values=q_values, p_values=p_values, n_values=n_values, length_values=length_values, classification_repeats=10, embedding_size=64)

clear_output(wait=True)
display(experiment_results_table)

,embedding size,p,q,accuracy (mean),accuracy (std),f1_micro (mean),f1_micro (std),f1_macro (mean),f1_macro (std)
0,64.0,0.1,0.1,0.609626,0.027500,0.609626,0.027500,0.576187,0.044834
1,64.0,0.3,0.1,0.593583,0.041575,0.593583,0.041575,0.573827,0.049040
2,64.0,0.5,0.1,0.596554,0.024585,0.596554,0.024585,0.562883,0.024755
3,64.0,0.7,0.1,0.651812,0.054034,0.651812,0.054034,0.619198,0.069280
4,64.0,0.9,0.1,0.622103,0.026439,0.622103,0.026439,0.596972,0.030680
5,64.0,1.0,0.1,0.610220,0.042366,0.610220,0.042366,0.579702,0.040723
6,64.0,3.0,0.1,0.614379,0.035868,0.614379,0.035868,0.583760,0.045510
7,64.0,5.0,0.1,0.635175,0.034452,0.635175,0.034452,0.601341,0.039992
8,64.0,7.0,0.1,0.628045,0.030051,0.628045,0.030051,0.586973,0.030312
9,64.0,9.0,0.1,0.609031,0.028122,0.609031,0.028122,0.589255,0.037117


link_classification: using 'dot' method to combine node embeddings into edge embeddings


KeyboardInterrupt: 

In [253]:
cora_stellar, cora_labels = get_cora_graph()

n_values = [100]
length_values = [5]
q_values = np.array([0.1, 0.3, 0.6, 1.0, 5.0, 10.0])
p_values = np.array([0.1, 0.3, 0.6, 1.0, 5.0, 10.0])
experiment_results_table = experiment(cora_stellar, cora_labels, q_values=q_values, p_values=p_values, n_values=n_values, length_values=length_values, classification_repeats=3, embedding_size=128, split=1)

clear_output(wait=True)
display(experiment_results_table)

,embedding size,p,q,accuracy (mean),accuracy (std),f1_micro (mean),f1_micro (std),f1_macro (mean),f1_macro (std)
0,128.0,0.1,0.1,0.810322,0.018097,0.810322,0.018097,0.785000,0.017708
1,128.0,0.3,0.1,0.796917,0.003351,0.796917,0.003351,0.785629,0.002885
2,128.0,0.6,0.1,0.793566,0.018767,0.793566,0.018767,0.775507,0.019198
3,128.0,1.0,0.1,0.808311,0.002681,0.808311,0.002681,0.794636,0.006345
4,128.0,5.0,0.1,0.788204,0.004021,0.788204,0.004021,0.770173,0.003809
5,128.0,10.0,0.1,0.794906,0.001340,0.794906,0.001340,0.778532,0.004001
6,128.0,0.1,0.3,0.776139,0.012064,0.776139,0.012064,0.763971,0.012381
7,128.0,0.3,0.3,0.781501,0.014745,0.781501,0.014745,0.765823,0.017729
8,128.0,0.6,0.3,0.799598,0.002011,0.799598,0.002011,0.777221,0.006220
9,128.0,1.0,0.3,0.820375,0.013405,0.820375,0.013405,0.792172,0.008481


In [252]:
experiment_results_table.to_excel('./cora-results-n100-l5.xlsx')
experiment_results_table.to_pickle('./cora-results-n100-l5.pkl')
experiment_results_table.to_csv('./cora-results-n100-l5.csv')
experiment_results_table.to_latex('./cora-results-n100-l5.tex')

/var/folders/rr/p4lmjn152vj75lt21_txptlh0000gn/T/ipykernel_40687/1254143326.py:4: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  experiment_results_table.to_latex('./cora-results-n20-l3.tex')


In [225]:
word2vec_results: Word2VecResult = word2vec_run(cora_stellar, n=20, length=3, p=0.5, q=0.5, embedding_size=64, verbose=1)
experiment_balanced_embeddings, experiment_balanced_labels = balance(word2vec_results.embeddings, cora_labels)

link_classification: using 'dot' method to combine node embeddings into edge embeddings
Epoch 1/2
3976/3976 [==============================] - 7s 2ms/step - loss: 0.6195 - binary_accuracy: 0.6349
Epoch 2/2
50/50 [==============================] - 0s 3ms/step
